In [112]:
# imports
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from time import sleep
from tqdm import tqdm

In [ ]:
# ジャンルとエリアの設定
GENRE = ""
AREA = ""
BASE_URL = f"https://www.mapion.co.jp/s/q={GENRE}%20{AREA}/t=spot/"

In [116]:
# 総ページ数を取得
response = requests.get(BASE_URL)
sleep(1)  # サーバーへの負荷を避けるために1秒待機
soup = BeautifulSoup(response.text, "html.parser")
total_pages = (int(soup.find_all("p", class_="subTitle")[0].find_all("span")[1].text.split('件')[0]) - 1 ) // 20 + 1
# スクレイピング可能な最大ページ数
max_pages = min(total_pages, 100)
# スクレイピングするURLリストの作成
urls = [f"{BASE_URL}p={page}" for page in range(1, max_pages + 1)]

In [ ]:
# スクレイピング開始
resurlts = []
for url in tqdm(urls):
    response = requests.get(url)
    sleep(1)  # サーバー負荷軽減のため1秒待機
    soup = BeautifulSoup(response.text, "html.parser")
    section = soup.find("div", id="NumberSection")
    cards = [f'{section.find_all("dl")[n]}' for n in list(range(len(section.find_all("dl")))) if n % 2 != 0]
    items = []
    for card_text in cards:
        card = BeautifulSoup(card_text, "html.parser")
        # 会社名を抽出
        name = " - "
        try:
            name = card.find('a').text.replace('\u3000', ' ').strip()
        except:
            name = " - "
        # 住所を抽出
        address = " - "
        try:
            address = card.find_all('dd')[2].text
        except:
            address = " - "
        # 電話番号を抽出
        phone = " - "
        try:
            phone = card.find_all('dd')[3].text.strip()
        except:
            phone = " - "
        items.append({
            "会社名": name,
            "住所": address,
            "電話番号": phone
        })
    resurlts.extend(items)

100%|██████████| 43/43 [00:52<00:00,  1.21s/it]


In [ ]:
# データフレームに変換してCSVファイルに出力
pd.DataFrame(resurlts).to_csv(f"data/{GENRE}_{AREA}.csv", index=False)